In [2]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dmagellan/')

In [3]:
from dmagellan.core.downsample import *

In [4]:
import pandas 
import string
#########
def preprocess_table(dataframe, idcol):
    strcols = list(get_str_cols(dataframe))
    strcols.append(idcol)
    projdf = dataframe[strcols]
    objsc = StringContainer()
    for row in projdf.itertuples():
        colvalues = row[1:-1]
        uid = row[-1]
        strings = [colvalue.strip() for colvalue in colvalues if not pd.isnull(colvalue)]
        concat_row = ' '.join(strings).lower()
#         print(concat_row)
        concat_row = concat_row.translate(string.punctuation)
        objsc.push_back(uid, str2bytes(concat_row))
    return objsc

In [5]:
def tokenize_strings(objsc, stopwords):
    n = objsc.size()
    objtc = TokenContainer()
    objtc.tokenize(objsc, stopwords)
    return objtc

In [6]:
def build_inv_index(objtc):
    inv_obj = InvertedIndex()
    inv_obj.build_inv_index_ds(objtc)
    return inv_obj

def probe(objtc, objinvindex, yparam):
    objprobe = Prober()
    objprobe.probe(objtc, objinvindex, yparam)
    return objprobe

In [7]:
def get_lrids(result_list):
    lids = set()
    rids = set()
    for i in range(len(result_list)):
        result = result_list[i]
        lids.update(result.get_lids())
        rids.update(result.get_rids())
    lids = sorted(lids)
    rids = sorted(rids)
    return [lids, rids]
    
    
def postprocess(lrids, ltable, rtable, lid, rid):
    s_ltable = ltable.map_partitions(lambda x: x[x[lid].isin(lrids[0])], meta=ltable)
    s_rtable = rtable.map_partitions(lambda x: x[x[rid].isin(lrids[1])], meta=rtable)
    return (s_ltable, s_rtable)
        

In [8]:
# pandas dask
import numpy as np
from dask import threaded
import dask.dataframe as dd
def sample_df(df, idcol, size):
    n = len(df)
    locs = np.random.choice(list(df[idcol].compute()), size)
    locs = list(set(locs))
    result = df.map_partitions(lambda x: x[x[idcol].isin(locs)])
    return result
    
def pandas_dask(ltable, rsample, lid, rid, size, yparam, stopwords, lchunks=None, rchunks=None, 
                scheduler=threaded.get):
    
    if lchunks is not None and ltable.npartitions > lchunks:
        ltable.repartition(lchunks)
    ltokens=[]
    for i in range(ltable.npartitions):
        lstrings = (delayed)(preprocess_table)(ltable.get_partition(i), lid)
        tokens = (delayed)(tokenize_strings)(lstrings, stopwords)
        ltokens.append(tokens)
    
    invindex = (delayed)(build_inv_index)(ltokens)
    
#     rsample = sample_df(rtable, rid, size)
    
#     if rchunks is not None and rsample.npartitions > rchunks:
#         rsample.repartition(rchunks)
    
    print(rsample.npartitions)
    probe_results = []
    for i in range(rsample.npartitions):
        rstrings = (delayed)(preprocess_table)(rsample.get_partition(i), rid)
        rtokens = (delayed)(tokenize_strings)(rstrings, stopwords)
        probe_result = (delayed)(probe)(rtokens, invindex, yparam)
        probe_results.append(probe_result)
    
    lr_ids = (delayed)(get_lrids)(probe_results)
    sampled_tables = (delayed)(postprocess)(lr_ids, ltable, rsample, lid, rid )
    x = dd.from_delayed(sampled_tables[0], meta=A)
    y = dd.from_delayed(sampled_tables[1], meta=B)
    return [x,y]
    
    

In [9]:
import os
def get_stop_words(path):
    stop_words_set = set()
    with open(path, "rb") as stopwords_file:
        for stop_words in stopwords_file:
            stop_words_set.add(stop_words.rstrip())

    return list(stop_words_set)

In [10]:
stopwords = get_stop_words('../../datasets/stopwords')
stopwords.extend(['the', 'my', 'i', 'andre', 'from', 'a', 'of', 'the', 'version', 'love', 'live', 'la', 'mix', 'album', \
                  'dont'])

In [11]:
# import dask.dataframe as dd

# A = dd.read_csv('../../datasets/tracks.csv', low_memory=False, blocksize=100000)
# B = dd.read_csv('../../datasets/songs.csv', low_memory=False, blocksize=100000)

# sampled_tables = pandas_dask(A, B, 'id', 'id', 10000, 1, stopwords=stopwords)

In [23]:
A = pd.read_csv('../../datasets/tracks.csv', low_memory=False, encoding='latin-1')
B = pd.read_csv('../../datasets/songs.csv', low_memory=False, encoding='latin-1')

In [24]:
ltable = dd.from_pandas(A, npartitions=4)

In [25]:
rs = B.head(10000)
rsample = dd.from_pandas(rs, npartitions=4)

In [15]:
len(ltable)

734485

In [26]:
sampled_tables = pandas_dask(ltable, rsample, 'id', 'id', 10000, 1, stopwords=stopwords)

4


In [27]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, ProgressBar
# with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof, ProgressBar():
with ProgressBar():
    result = sampled_tables[0].compute()

[#############                           ] | 34% Completed | 23.1s


TypeError: character mapping must return integer, None or unicode

Traceback
---------
  File "/Users/pradap/miniconda2/lib/python2.7/site-packages/dask/async.py", line 266, in execute_task
    result = _execute_task(task, data)
  File "/Users/pradap/miniconda2/lib/python2.7/site-packages/dask/async.py", line 247, in _execute_task
    return func(*args2)
  File "<ipython-input-4-7c60cab96ef4>", line 15, in preprocess_table
    concat_row = concat_row.translate(string.punctuation)


In [14]:
lrids = lr

pandas.core.frame.DataFrame

In [101]:
prof.visualize()

Figure(id='490d30c4-2b96-4e96-ad74-512bdd0ac251', ...)